In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def f(x):
  return 3*x**2 - 4*x + 5

In [ ]:
f(3.0)

In [ ]:
xs = np.arange(-5, 5, 0.25)
ys = f(xs)
plt.plot(xs,ys)

In [ ]:
h = 0.00001
x = 2/3
f(x+h)

# So how do we update "Value" to track the differentiation steps?
## We assign a new item called $gradient$ or `grad`
## and we add a `backward()` method to calculate the gradient at each step

## So where do we start?
### recall $d(x)/d(x) = 1$ always
#### So our final value of our expression starts with its .grad set to 1.0
### What's the next step?
#### Assume the expression was L = a + b (a = 2, b=3), so L = 5
 - $d(L)/d(L) = 1$
 - $d(L)/d(a) = d(L)/d(L) * d(L)/d(a) = 1.0 $
 - $d(L)/d(b) = d(L)/d(L) * d(L)/d(b) = 1.0 $
#### Assume expression was L = a*b (a=2, b=3), so L = 6
 - $d(L)/d(L) = 1$
 - $d(L)/d(a) = d(L)/d(L) * d(L)/d(a) = 1.0*b = 3 $
 - $d(L)/d(b) = d(L)/d(L) * d(L)/d(b) = 1.0*a = 2 $
#### Assume expression was L = pow(a,b) (a=2,b=3) so L = 8
 - $d(L)/d(L) = 1$
 - $d(L)/d(a) = b*a^{b-1}$
 - $d(L)/d(a) = d(L)/d(L) * d(L)/d(a) = (1*)b*a^{b-1}) = (1*)3*((2)^2) = 12$

In [ ]:
class Value:
  def __init__(s, data, _c = (), _op = '', label = ''):
    s.data = data
    s.grad = 0.0
    s._backward = lambda : None
    s._prev = set(_c)
    s._op = _op
    s.label = label
  def __repr__(s):
    return f'V(data={s.data} {s.label} {s._op})'
  # when variables are used multiple times, the grad field is overwritten
  # so the solution is to accumulate the gradients
  def __add__(s, o):
    o = o if isinstance(o, Value) else Value(o)
    out = Value(s.data + o.data, (s, o), '+')
    def _backward():
      s.grad += 1.0 * out.grad
      o.grad += 1.0 * out.grad
    out._backward = _backward
    return out
  def __mul__(s, o):
    o = o if isinstance(o, Value) else Value(o)
    out = Value(s.data * o.data, (s, o), '*')
    def _backward():
      s.grad += o.data * out.grad
      o.grad += s.data * out.grad
    out._backward = _backward
    return out
  def __pow__(s, o):
    assert isinstance(o, (int, float)), "only supporting ints and floats for now"
    out = Value(s.data ** o, (s,), f'**{o}')
    def _backward():
      s.grad += o * ((s.data ** (o - 1))) * out.grad 
    out._backward = _backward
    return out
  def exp(s):
    x = s.data
    out = Value(math.exp(x), (s,), 'exp')
    def _backward():
      s.grad += out.data * out.grad
    out._backward = _backward
    return out

  def tanh(s):
    x = s.data
    t = (math.exp(2*x) - 1)/(math.exp(2*x)+1)
    out = Value(t, (s,), 'tanh')
    def _backward():
      s.grad += (1 - t**2) * out.grad
      # print(f" grad:{s.grad} {t} {out.grad}")
    out._backward = _backward 
    return out
  def __rmul__(s, o):
    return s * o
  def __truediv__(s, o):
    return s * (o ** -1)
  def __radd__(s, o):
    return s + o
  def __neg__(s):
    return s * -1
  def __sub__(s, o):
    return s + (-o)

  def backward(s):
    topo = []
    visited = set()
    def dfs(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          dfs(child)
        topo.append(v)
    dfs(s)
    s.grad = 1.0
    for node in reversed(topo):
      #print('calling', node, '_backward()')
      node._backward()

In [ ]:
from graphviz import Digraph
def trace(r):
  # builds a set of all nodes and edges in a graph
  nodes, edges = set(), set()
  def build(v):
    if v not in nodes:
      nodes.add(v)
      for child in v._prev:
        edges.add((child, v))
        build(child)
  build(r)
  return nodes, edges
def draw_dot(r):
  dot = Digraph(format='svg', graph_attr={'rankdir' : 'LR'}) # LR = left to right
  nodes, edges = trace(r)
  for n in nodes:
    uid = str(id(n))
    dot.node(name = uid, label = "{ %s | data %.4f | grad %.4f }" % (n.label, n.data, n.grad), shape='record')
    if n._op:
      dot.node(name = uid + n._op, label = n._op)
      dot.edge(uid + n._op, uid)
  for n1, n2 in edges:
    dot.edge(str(id(n1)), str(id(n2)) + n2._op)
  return dot

In [ ]:
# L = Value(2.0, label='a') + Value(3.0, label='b')
# L = Value(2.0, label='a') * Value(3.0, label='b')
# L = Value(2.0, label='a') ** 3.0
# L.backward()
# draw_dot(L)

In [ ]:
a = Value(2.0, label='a')
b = Value(-3.00, label='b')
c = Value(10.0, label='c')
# d = a*b + c
e = a*b; e.label = 'e'
d = e + c; d.label = 'd'
# d = a*b + c
print(f'{d=} is the value we calculate earlier. Now here is some more expressions')
f = Value(-2.0, label = 'f')
L = d*f
L.label = 'L'
print(f'final value {L=}')

In [ ]:
# a = Value(2.)
# b = Value(4.0)
# a/ b
# a * (1/b)
# a * (b**-1)

In [ ]:
L.backward()
draw_dot(L)

In [ ]:
L = d * f; L.label='L'
# dl/dd =? f
# (f(x+h) - f(x))/h
# ((d+h)*f - d*f)/h
# d*f + h* f - d*f)/h
# (h*f)/g
# f

In [ ]:
def simple1():
  h = 0.0001
  a = Value(2.0, label='a')
  b = Value(-3.00, label='b')
  c = Value(10.0, label='c')
  e = a*b; e.label = 'e'
  d = e + c; d.label = 'd'
  f = Value(-2.0, label = 'f')
  L = d*f
  L.label = 'L'
  L1 = L.data

  a = Value(2.0, label='a')
  b = Value(-3.0, label='b')
  c = Value(10.0, label='c')
  e = a*b; e.label = 'e'
  d = e + c; d.label = 'd'
  f = Value(-2.0, label = 'f')
  L = d*f
  # L.data += h
  L2 = L.data
  if L2 != L1:
    print(f'{L1=} {L2=}  {(L2 - L1)/h=}')
  else:
    print(f'{L=}')
simple1()

In [ ]:
#L.grad = 1.0
#f.grad = 4.0
#d.grad = -2
#e.grad = -2
#c.grad = -2
#b.grad = -4
#a.grad = 6

In [ ]:
a = Value(-2.0, label='a')
b = Value(3.0, label = 'b')
d = a * b; d.label = 'd'
e = a + b; e.label = 'e'
f = d * e; f.label = 'f'

draw_dot(f)

In [ ]:
f.backward()
draw_dot(f)

In [ ]:
plt.plot(np.arange(-5,5, 0.2), np.tanh(np.arange(-5,5,0.2))); plt.grid();

In [ ]:
# inputs x1, x2
x1 = Value(2.0, label='x1')
x2 = Value(0.0, label='x2')
# weights
w1 = Value(-3.0, label='w1')
w2 = Value(1.0, label='w2')
# bias of the neuron
bb = Value(6.8813735870195432, label='b')
x1w1 = x1*w1; x1w1.label = 'x1*w1'
x2w2 = x2*w2; x2w2.label = 'x2*w2'
x1w1x2w2 = x1w1 + x2w2; x1w1x2w2.label='x1*w1 + x2*w2'
n = x1w1x2w2 + bb; n.label = 'n'
o = n.tanh(); o.label = 'o'
o.backward()
draw_dot(o)

In [ ]:
# inputs x1, x2data:image/svg+xml,%3C%3Fxml%20version%3D%221.0%22%20encoding%3D%22UTF-8%22%20standalone%3D%22no%22%3F%3E%0A%3C!DOCTYPE%20svg%20PUBLIC%20%22-%2F%2FW3C%2F%2FDTD%20SVG%201.1%2F%2FEN%22%0A%20%22http%3A%2F%2Fwww.w3.org%2FGraphics%2FSVG%2F1.1%2FDTD%2Fsvg11.dtd%22%3E%0A%3C!--%20Generated%20by%20graphviz%20version%202.48.0%20(0)%0A%20--%3E%0A%3C!--%20Pages%3A%201%20--%3E%0A%3Csvg%20width%3D%221580pt%22%20height%3D%22210pt%22%0A%20viewBox%3D%220.00%200.00%201580.00%20210.00%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20xmlns%3Axlink%3D%22http%3A%2F%2Fwww.w3.org%2F1999%2Fxlink%22%3E%0A%3Cg%20id%3D%22graph0%22%20class%3D%22graph%22%20transform%3D%22scale(1%201)%20rotate(0)%20translate(4%20206)%22%3E%0A%3Cpolygon%20fill%3D%22white%22%20stroke%3D%22transparent%22%20points%3D%22-4%2C4%20-4%2C-206%201576%2C-206%201576%2C4%20-4%2C4%22%2F%3E%0A%3C!--%20139917773591568%20--%3E%0A%3Cg%20id%3D%22node1%22%20class%3D%22node%22%3E%0A%3Ctitle%3E139917773591568%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22black%22%20points%3D%22327%2C-55.5%20327%2C-91.5%20548%2C-91.5%20548%2C-55.5%20327%2C-55.5%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22353.5%22%20y%3D%22-69.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%3Ex1*w1%3C%2Ftext%3E%0A%3Cpolyline%20fill%3D%22none%22%20stroke%3D%22black%22%20points%3D%22380%2C-55.5%20380%2C-91.5%20%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22422.5%22%20y%3D%22-69.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%3Edata%20%26%2345%3B6.0000%3C%2Ftext%3E%0A%3Cpolyline%20fill%3D%22none%22%20stroke%3D%22black%22%20points%3D%22465%2C-55.5%20465%2C-91.5%20%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22506.5%22%20y%3D%22-69.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%3Egrad%200.5000%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%20139918157674320%2B%20--%3E%0A%3Cg%20id%3D%22node12%22%20class%3D%22node%22%3E%0A%3Ctitle%3E139918157674320%2B%3C%2Ftitle%3E%0A%3Cellipse%20fill%3D%22none%22%20stroke%3D%22black%22%20cx%3D%22611%22%20cy%3D%22-100.5%22%20rx%3D%2227%22%20ry%3D%2218%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22611%22%20y%3D%22-96.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%3E%2B%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%20139917773591568%26%2345%3B%26gt%3B139918157674320%2B%20--%3E%0A%3Cg%20id%3D%22edge10%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E139917773591568%26%2345%3B%26gt%3B139918157674320%2B%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22black%22%20d%3D%22M548.08%2C-90.75C557.49%2C-92.23%20566.48%2C-93.65%20574.54%2C-94.92%22%2F%3E%0A%3Cpolygon%20fill%3D%22black%22%20stroke%3D%22black%22%20points%3D%22574.16%2C-98.4%20584.58%2C-96.5%20575.25%2C-91.49%20574.16%2C-98.4%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%20139917773591568*%20--%3E%0A%3Cg%20id%3D%22node2%22%20class%3D%22node%22%3E%0A%3Ctitle%3E139917773591568*%3C%2Ftitle%3E%0A%3Cellipse%20fill%3D%22none%22%20stroke%3D%22black%22%20cx%3D%22264%22%20cy%3D%22-73.5%22%20rx%3D%2227%22%20ry%3D%2218%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22264%22%20y%3D%22-69.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%3E*%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%20139917773591568*%26%2345%3B%26gt%3B139917773591568%20--%3E%0A%3Cg%20id%3D%22edge1%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E139917773591568*%26%2345%3B%26gt%3B139917773591568%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22black%22%20d%3D%22M291.05%2C-73.5C298.48%2C-73.5%20307.1%2C-73.5%20316.36%2C-73.5%22%2F%3E%0A%3Cpolygon%20fill%3D%22black%22%20stroke%3D%22black%22%20points%3D%22316.64%2C-77%20326.64%2C-73.5%20316.64%2C-70%20316.64%2C-77%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%20139917773593616%20--%3E%0A%3Cg%20id%3D%22node3%22%20class%3D%22node%22%3E%0A%3Ctitle%3E139917773593616%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22black%22%20points%3D%224%2C-165.5%204%2C-201.5%20197%2C-201.5%20197%2C-165.5%204%2C-165.5%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%2219%22%20y%3D%22-179.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%3Ex2%3C%2Ftext%3E%0A%3Cpolyline%20fill%3D%22none%22%20stroke%3D%22black%22%20points%3D%2234%2C-165.5%2034%2C-201.5%20%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%2274%22%20y%3D%22-179.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%3Edata%200.0000%3C%2Ftext%3E%0A%3Cpolyline%20fill%3D%22none%22%20stroke%3D%22black%22%20points%3D%22114%2C-165.5%20114%2C-201.5%20%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22155.5%22%20y%3D%22-179.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%3Egrad%200.5000%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%20139917773589968*%20--%3E%0A%3Cg%20id%3D%22node15%22%20class%3D%22node%22%3E%0A%3Ctitle%3E139917773589968*%3C%2Ftitle%3E%0A%3Cellipse%20fill%3D%22none%22%20stroke%3D%22black%22%20cx%3D%22264%22%20cy%3D%22-128.5%22%20rx%3D%2227%22%20ry%3D%2218%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22264%22%20y%3D%22-124.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%3E*%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%20139917773593616%26%2345%3B%26gt%3B139917773589968*%20--%3E%0A%3Cg%20id%3D%22edge11%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E139917773593616%26%2345%3B%26gt%3B139917773589968*%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22black%22%20d%3D%22M173.4%2C-165.46C182.75%2C-162.68%20192.16%2C-159.67%20201%2C-156.5%20211.53%2C-152.72%20222.76%2C-147.9%20232.72%2C-143.34%22%2F%3E%0A%3Cpolygon%20fill%3D%22black%22%20stroke%3D%22black%22%20points%3D%22234.25%2C-146.48%20241.83%2C-139.07%20231.29%2C-140.14%20234.25%2C-146.48%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%20139917810380880%20--%3E%0A%3Cg%20id%3D%22node4%22%20class%3D%22node%22%3E%0A%3Ctitle%3E139917810380880%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22black%22%20points%3D%22718%2C-137.5%20718%2C-173.5%20904%2C-173.5%20904%2C-137.5%20718%2C-137.5%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22729.5%22%20y%3D%22-151.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%3Eb%3C%2Ftext%3E%0A%3Cpolyline%20fill%3D%22none%22%20stroke%3D%22black%22%20points%3D%22741%2C-137.5%20741%2C-173.5%20%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22781%22%20y%3D%22-151.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%3Edata%206.8814%3C%2Ftext%3E%0A%3Cpolyline%20fill%3D%22none%22%20stroke%3D%22black%22%20points%3D%22821%2C-137.5%20821%2C-173.5%20%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22862.5%22%20y%3D%22-151.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%3Egrad%200.5000%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%20139917807855888%2B%20--%3E%0A%3Cg%20id%3D%22node9%22%20class%3D%22node%22%3E%0A%3Ctitle%3E139917807855888%2B%3C%2Ftitle%3E%0A%3Cellipse%20fill%3D%22none%22%20stroke%3D%22black%22%20cx%3D%221011%22%20cy%3D%22-127.5%22%20rx%3D%2227%22%20ry%3D%2218%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221011%22%20y%3D%22-123.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%3E%2B%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%20139917810380880%26%2345%3B%26gt%3B139917807855888%2B%20--%3E%0A%3Cg%20id%3D%22edge7%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E139917810380880%26%2345%3B%26gt%3B139917807855888%2B%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22black%22%20d%3D%22M904.04%2C-142.48C929.04%2C-138.95%20954.63%2C-135.33%20974.4%2C-132.53%22%2F%3E%0A%3Cpolygon%20fill%3D%22black%22%20stroke%3D%22black%22%20points%3D%22975.05%2C-135.98%20984.46%2C-131.11%20974.07%2C-129.05%20975.05%2C-135.98%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%20139917809751120%20--%3E%0A%3Cg%20id%3D%22node5%22%20class%3D%22node%22%3E%0A%3Ctitle%3E139917809751120%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22black%22%20points%3D%221386%2C-109.5%201386%2C-145.5%201572%2C-145.5%201572%2C-109.5%201386%2C-109.5%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221397.5%22%20y%3D%22-123.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%3Eo%3C%2Ftext%3E%0A%3Cpolyline%20fill%3D%22none%22%20stroke%3D%22black%22%20points%3D%221409%2C-109.5%201409%2C-145.5%20%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221449%22%20y%3D%22-123.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%3Edata%200.7071%3C%2Ftext%3E%0A%3Cpolyline%20fill%3D%22none%22%20stroke%3D%22black%22%20points%3D%221489%2C-109.5%201489%2C-145.5%20%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221530.5%22%20y%3D%22-123.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%3Egrad%201.0000%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%20139917809751120tanh%20--%3E%0A%3Cg%20id%3D%22node6%22%20class%3D%22node%22%3E%0A%3Ctitle%3E139917809751120tanh%3C%2Ftitle%3E%0A%3Cellipse%20fill%3D%22none%22%20stroke%3D%22black%22%20cx%3D%221323%22%20cy%3D%22-127.5%22%20rx%3D%2227%22%20ry%3D%2218%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221323%22%20y%3D%22-123.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%3Etanh%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%20139917809751120tanh%26%2345%3B%26gt%3B139917809751120%20--%3E%0A%3Cg%20id%3D%22edge2%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E139917809751120tanh%26%2345%3B%26gt%3B139917809751120%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22black%22%20d%3D%22M1350.12%2C-127.5C1357.62%2C-127.5%201366.29%2C-127.5%201375.5%2C-127.5%22%2F%3E%0A%3Cpolygon%20fill%3D%22black%22%20stroke%3D%22black%22%20points%3D%221375.69%2C-131%201385.69%2C-127.5%201375.69%2C-124%201375.69%2C-131%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%20139917773164176%20--%3E%0A%3Cg%20id%3D%22node7%22%20class%3D%22node%22%3E%0A%3Ctitle%3E139917773164176%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22black%22%20points%3D%220%2C-55.5%200%2C-91.5%20201%2C-91.5%20201%2C-55.5%200%2C-55.5%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%2216.5%22%20y%3D%22-69.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%3Ew1%3C%2Ftext%3E%0A%3Cpolyline%20fill%3D%22none%22%20stroke%3D%22black%22%20points%3D%2233%2C-55.5%2033%2C-91.5%20%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%2275.5%22%20y%3D%22-69.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%3Edata%20%26%2345%3B3.0000%3C%2Ftext%3E%0A%3Cpolyline%20fill%3D%22none%22%20stroke%3D%22black%22%20points%3D%22118%2C-55.5%20118%2C-91.5%20%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22159.5%22%20y%3D%22-69.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%3Egrad%201.0000%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%20139917773164176%26%2345%3B%26gt%3B139917773591568*%20--%3E%0A%3Cg%20id%3D%22edge13%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E139917773164176%26%2345%3B%26gt%3B139917773591568*%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22black%22%20d%3D%22M201.02%2C-73.5C210.04%2C-73.5%20218.72%2C-73.5%20226.59%2C-73.5%22%2F%3E%0A%3Cpolygon%20fill%3D%22black%22%20stroke%3D%22black%22%20points%3D%22226.77%2C-77%20236.77%2C-73.5%20226.77%2C-70%20226.77%2C-77%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%20139917807855888%20--%3E%0A%3Cg%20id%3D%22node8%22%20class%3D%22node%22%3E%0A%3Ctitle%3E139917807855888%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22black%22%20points%3D%221074%2C-109.5%201074%2C-145.5%201260%2C-145.5%201260%2C-109.5%201074%2C-109.5%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221085.5%22%20y%3D%22-123.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%3En%3C%2Ftext%3E%0A%3Cpolyline%20fill%3D%22none%22%20stroke%3D%22black%22%20points%3D%221097%2C-109.5%201097%2C-145.5%20%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221137%22%20y%3D%22-123.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%3Edata%200.8814%3C%2Ftext%3E%0A%3Cpolyline%20fill%3D%22none%22%20stroke%3D%22black%22%20points%3D%221177%2C-109.5%201177%2C-145.5%20%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%221218.5%22%20y%3D%22-123.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%3Egrad%200.5000%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%20139917807855888%26%2345%3B%26gt%3B139917809751120tanh%20--%3E%0A%3Cg%20id%3D%22edge8%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E139917807855888%26%2345%3B%26gt%3B139917809751120tanh%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22black%22%20d%3D%22M1260.25%2C-127.5C1269.15%2C-127.5%201277.76%2C-127.5%201285.59%2C-127.5%22%2F%3E%0A%3Cpolygon%20fill%3D%22black%22%20stroke%3D%22black%22%20points%3D%221285.73%2C-131%201295.73%2C-127.5%201285.73%2C-124%201285.73%2C-131%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%20139917807855888%2B%26%2345%3B%26gt%3B139917807855888%20--%3E%0A%3Cg%20id%3D%22edge3%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E139917807855888%2B%26%2345%3B%26gt%3B139917807855888%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22black%22%20d%3D%22M1038.12%2C-127.5C1045.62%2C-127.5%201054.29%2C-127.5%201063.5%2C-127.5%22%2F%3E%0A%3Cpolygon%20fill%3D%22black%22%20stroke%3D%22black%22%20points%3D%221063.69%2C-131%201073.69%2C-127.5%201063.69%2C-124%201063.69%2C-131%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%20139917810381584%20--%3E%0A%3Cg%20id%3D%22node10%22%20class%3D%22node%22%3E%0A%3Ctitle%3E139917810381584%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22black%22%20points%3D%222.5%2C-110.5%202.5%2C-146.5%20198.5%2C-146.5%20198.5%2C-110.5%202.5%2C-110.5%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%2219%22%20y%3D%22-124.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%3Ew2%3C%2Ftext%3E%0A%3Cpolyline%20fill%3D%22none%22%20stroke%3D%22black%22%20points%3D%2235.5%2C-110.5%2035.5%2C-146.5%20%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%2275.5%22%20y%3D%22-124.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%3Edata%201.0000%3C%2Ftext%3E%0A%3Cpolyline%20fill%3D%22none%22%20stroke%3D%22black%22%20points%3D%22115.5%2C-110.5%20115.5%2C-146.5%20%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22157%22%20y%3D%22-124.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%3Egrad%200.0000%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%20139917810381584%26%2345%3B%26gt%3B139917773589968*%20--%3E%0A%3Cg%20id%3D%22edge12%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E139917810381584%26%2345%3B%26gt%3B139917773589968*%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22black%22%20d%3D%22M198.71%2C-128.5C208.57%2C-128.5%20218.09%2C-128.5%20226.65%2C-128.5%22%2F%3E%0A%3Cpolygon%20fill%3D%22black%22%20stroke%3D%22black%22%20points%3D%22226.92%2C-132%20236.92%2C-128.5%20226.92%2C-125%20226.92%2C-132%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%20139918157674320%20--%3E%0A%3Cg%20id%3D%22node11%22%20class%3D%22node%22%3E%0A%3Ctitle%3E139918157674320%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22black%22%20points%3D%22674%2C-82.5%20674%2C-118.5%20948%2C-118.5%20948%2C-82.5%20674%2C-82.5%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22727%22%20y%3D%22-96.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%3Ex1*w1%20%2B%20x2*w2%3C%2Ftext%3E%0A%3Cpolyline%20fill%3D%22none%22%20stroke%3D%22black%22%20points%3D%22780%2C-82.5%20780%2C-118.5%20%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22822.5%22%20y%3D%22-96.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%3Edata%20%26%2345%3B6.0000%3C%2Ftext%3E%0A%3Cpolyline%20fill%3D%22none%22%20stroke%3D%22black%22%20points%3D%22865%2C-82.5%20865%2C-118.5%20%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22906.5%22%20y%3D%22-96.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%3Egrad%200.5000%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%20139918157674320%26%2345%3B%26gt%3B139917807855888%2B%20--%3E%0A%3Cg%20id%3D%22edge14%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E139918157674320%26%2345%3B%26gt%3B139917807855888%2B%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22black%22%20d%3D%22M944.07%2C-118.51C954.98%2C-120%20965.29%2C-121.4%20974.36%2C-122.64%22%2F%3E%0A%3Cpolygon%20fill%3D%22black%22%20stroke%3D%22black%22%20points%3D%22974.04%2C-126.13%20984.42%2C-124.01%20974.99%2C-119.19%20974.04%2C-126.13%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%20139918157674320%2B%26%2345%3B%26gt%3B139918157674320%20--%3E%0A%3Cg%20id%3D%22edge4%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E139918157674320%2B%26%2345%3B%26gt%3B139918157674320%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22black%22%20d%3D%22M638.29%2C-100.5C645.71%2C-100.5%20654.34%2C-100.5%20663.71%2C-100.5%22%2F%3E%0A%3Cpolygon%20fill%3D%22black%22%20stroke%3D%22black%22%20points%3D%22663.72%2C-104%20673.72%2C-100.5%20663.72%2C-97%20663.72%2C-104%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%20139917773590864%20--%3E%0A%3Cg%20id%3D%22node13%22%20class%3D%22node%22%3E%0A%3Ctitle%3E139917773590864%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22black%22%20points%3D%222%2C-0.5%202%2C-36.5%20199%2C-36.5%20199%2C-0.5%202%2C-0.5%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%2217%22%20y%3D%22-14.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%3Ex1%3C%2Ftext%3E%0A%3Cpolyline%20fill%3D%22none%22%20stroke%3D%22black%22%20points%3D%2232%2C-0.5%2032%2C-36.5%20%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%2272%22%20y%3D%22-14.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%3Edata%202.0000%3C%2Ftext%3E%0A%3Cpolyline%20fill%3D%22none%22%20stroke%3D%22black%22%20points%3D%22112%2C-0.5%20112%2C-36.5%20%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22155.5%22%20y%3D%22-14.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%3Egrad%20%26%2345%3B1.5000%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%20139917773590864%26%2345%3B%26gt%3B139917773591568*%20--%3E%0A%3Cg%20id%3D%22edge9%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E139917773590864%26%2345%3B%26gt%3B139917773591568*%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22black%22%20d%3D%22M170.4%2C-36.57C180.73%2C-39.66%20191.2%2C-43%20201%2C-46.5%20211.28%2C-50.17%20222.28%2C-54.73%20232.11%2C-59.06%22%2F%3E%0A%3Cpolygon%20fill%3D%22black%22%20stroke%3D%22black%22%20points%3D%22230.93%2C-62.37%20241.48%2C-63.27%20233.79%2C-55.98%20230.93%2C-62.37%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%20139917773589968%20--%3E%0A%3Cg%20id%3D%22node14%22%20class%3D%22node%22%3E%0A%3Ctitle%3E139917773589968%3C%2Ftitle%3E%0A%3Cpolygon%20fill%3D%22none%22%20stroke%3D%22black%22%20points%3D%22329.5%2C-110.5%20329.5%2C-146.5%20545.5%2C-146.5%20545.5%2C-110.5%20329.5%2C-110.5%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22356%22%20y%3D%22-124.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%3Ex2*w2%3C%2Ftext%3E%0A%3Cpolyline%20fill%3D%22none%22%20stroke%3D%22black%22%20points%3D%22382.5%2C-110.5%20382.5%2C-146.5%20%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22422.5%22%20y%3D%22-124.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%3Edata%200.0000%3C%2Ftext%3E%0A%3Cpolyline%20fill%3D%22none%22%20stroke%3D%22black%22%20points%3D%22462.5%2C-110.5%20462.5%2C-146.5%20%22%2F%3E%0A%3Ctext%20text-anchor%3D%22middle%22%20x%3D%22504%22%20y%3D%22-124.8%22%20font-family%3D%22Times%2Cserif%22%20font-size%3D%2214.00%22%3Egrad%200.5000%3C%2Ftext%3E%0A%3C%2Fg%3E%0A%3C!--%20139917773589968%26%2345%3B%26gt%3B139918157674320%2B%20--%3E%0A%3Cg%20id%3D%22edge6%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E139917773589968%26%2345%3B%26gt%3B139918157674320%2B%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22black%22%20d%3D%22M545.65%2C-111.01C555.95%2C-109.32%20565.81%2C-107.71%20574.59%2C-106.28%22%2F%3E%0A%3Cpolygon%20fill%3D%22black%22%20stroke%3D%22black%22%20points%3D%22575.4%2C-109.7%20584.7%2C-104.63%20574.27%2C-102.79%20575.4%2C-109.7%22%2F%3E%0A%3C%2Fg%3E%0A%3C!--%20139917773589968*%26%2345%3B%26gt%3B139917773589968%20--%3E%0A%3Cg%20id%3D%22edge5%22%20class%3D%22edge%22%3E%0A%3Ctitle%3E139917773589968*%26%2345%3B%26gt%3B139917773589968%3C%2Ftitle%3E%0A%3Cpath%20fill%3D%22none%22%20stroke%3D%22black%22%20d%3D%22M291.05%2C-128.5C299.21%2C-128.5%20308.81%2C-128.5%20319.12%2C-128.5%22%2F%3E%0A%3Cpolygon%20fill%3D%22black%22%20stroke%3D%22black%22%20points%3D%22319.2%2C-132%20329.2%2C-128.5%20319.2%2C-125%20319.2%2C-132%22%2F%3E%0A%3C%2Fg%3E%0A%3C%2Fg%3E%0A%3C%2Fsvg%3E%0A
x1 = Value(2.0, label='x1')
x2 = Value(0.0, label='x2')
# weights
w1 = Value(-3.0, label='w1')
w2 = Value(1.0, label='w2')
# bias of the neuron
bb = Value(6.8813735870195432, label='b')
x1w1 = x1*w1; x1w1.label = 'x1*w1'
x2w2 = x2*w2; x2w2.label = 'x2*w2'
x1w1x2w2 = x1w1 + x2w2; x1w1x2w2.label='x1*w1 + x2*w2'
n = x1w1x2w2 + bb; n.label = 'n'
# o = n.tanh(); o.label = 'o'
e = (2*n).exp()
o = (e-1)/(e+1)
o.backward()
draw_dot(o)

In [ ]:
import torch
x1 = torch.Tensor([2.0]).double()   ; x1.requires_grad = True
x2 = torch.Tensor([0.0]).double()   ; x2.requires_grad = True 
w1 = torch.Tensor([-3.0]).double()  ; w1.requires_grad = True
w2 = torch.Tensor([1.0]).double()   ; w2.requires_grad = True
b = torch.Tensor([6.8813735870195432]).double() ; b.requires_grad = True
n = x1 * w1 + x2 * w2 + b
o = torch.tanh(n)

print(o.data.item())
o.backward()

print('-----')
print('x1.grad', x1.grad.item())
print('x2.grad', x2.grad.item())
print('w1.grad', w1.grad.item())
print('w2.grad', w2.grad.item())


In [ ]:
import random
class Neuron:
  def __init__(s, nin):
    s.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
    s.b = Value(random.uniform(-1,1))
  def __call__(s, x):
    # w * x + b
    act = sum((wi * xi for wi, xi in zip(s.w, x)), s.b)
    out = act.tanh()
    return out
  def parameters(s):
    return s.w + [s.b]
class Layer:
  def __init__(s, nin, nout):
    s.neurons = [ Neuron(nin) for _ in range(nout) ]
  def __call__(s, x):
    outs = [ n(x) for n in s.neurons ]
    return outs
  def parameters(s):
    return[ p for neuron in s.neurons for p in neuron.parameters() ]

class MLP:
  def __init__(s, nin, nouts):
    """ nouts in a list of sizes for the individual layers """
    sz = [nin] + nouts
    s.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]
  def __call__(s,x):
    for layer in s.layers:
      x = layer(x)
    return x[0] if len(x) == 1 else x
  def parameters(s):
    return [ p for layer in s.layers for p in layer.parameters() ]
            


In [ ]:
xs = [
  [2.0, 3.0, -1.0],
  [3.0, -1.0, 0.5],
  [0.5, 1.0, 1.0],
  [1.0, 1.0, -1.0],
]
ys = [ 1.0, -1.0, -1.0, 1.0] # desired targets
np.random.seed(1337)
random.seed(1337)

x = [2.0, 3.0, -1.0]
n = MLP(3, [4, 4, 1])
z = n(x)
print('number of params', len(n.parameters()))
draw_dot(z)

In [ ]:
print('num params', len(n.parameters()))
ypred = [ n(x) for x in xs ]
ypred

In [ ]:
ypred = [ n(x) for x in xs ]
print(ypred)
loss = sum([(yout - ygt)**2 for ygt, yout in zip(ys, ypred)])
loss.backward()
loss

In [ ]:
print(loss)
draw_dot(loss)

In [ ]:
[ (p.data, p.grad) for p in n.parameters() ]

In [ ]:
for p in n.parameters():
  p.data += -0.01 * p.grad
  p.grad = 0.0

In [ ]:
ypred = [ n(x) for x in xs ]
loss = sum([(yout - ygt)**2 for ygt, yout in zip(ys, ypred)])
loss.backward()
print(loss)

In [ ]:
for p in n.parameters():
  p.data += -0.01 * p.grad
  p.grad = 0.0

ypred = [ n(x) for x in xs ]
loss = sum([(yout - ygt)**2 for ygt, yout in zip(ys, ypred)])
loss.backward()
print(loss)

In [ ]:
for p in n.parameters():
  p.data += -0.01 * p.grad
  p.grad = 0.0

ypred = [ n(x) for x in xs ]
loss = sum([(yout - ygt)**2 for ygt, yout in zip(ys, ypred)])
loss.backward()
print(loss)

In [ ]:
for p in n.parameters():
  p.data += -0.01 * p.grad
  p.grad = 0.0
ypred = [ n(x) for x in xs ]
loss = sum([(yout - ygt)**2 for ygt, yout in zip(ys, ypred)])
loss.backward()
print(loss)

In [ ]:
np.random.seed(1337)
random.seed(1337)

xs = [
  [2.0, 3.0, -1.0],
  [3.0, -1.0, 0.5],
  [0.5, 1.0, 1.0],
  [1.0, 1.0, -1.0],
]
x = [2.0, 3.0, -1.0]
n = MLP(3, [4, 4, 1])
z = n(x)
n.parameters()
print('num params', len(n.parameters()))
ys = [ 1.0, -1.0, -1.0, 1.0] # desired targets



In [ ]:
for k in range(20):
  # forward pass
  ypred = [n(x) for x in xs ]
  loss = sum((yout-ygt)**2 for ygt, yout in zip(ys, ypred))
  # backward pass
  for p in n.parameters():
    p.grad = 0.0
  loss.backward()
  # update
  for p in n.parameters():
    p.data += -0.01 *p.grad
  print(k, loss.data)